In [4]:
import asyncio
import time
from http import HTTPStatus
from typing import Dict, List, Optional, Tuple, Any, Union, Set, Callable

import aiohttp
import requests
from loguru import logger


class RequestHandler:
    """
    Handles HTTP requests with retry logic and error handling.
    
    This class encapsulates the HTTP request functionality, providing
    both synchronous and asynchronous request methods with retry logic.
    """
    
    def __init__(
        self, 
        base_url: str,
        default_timeout: int = 30,
        long_timeout: int = 300,
        max_retries: int = 5,
        retry_delay: int = 5,
        known_error_messages: Optional[List[str]] = None,
        verify_ssl: bool = False
    ):
        """
        Initialize the request handler.
        
        Args:
            base_url: Base URL for all requests
            default_timeout: Default timeout for requests in seconds
            long_timeout: Timeout for long-running requests in seconds
            max_retries: Maximum number of retry attempts
            retry_delay: Delay between retries in seconds
            known_error_messages: List of error message patterns to detect in responses
            verify_ssl: Whether to verify SSL certificates
        """
        self.base_url = base_url
        self.default_timeout = default_timeout
        self.long_timeout = long_timeout
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.known_error_messages = known_error_messages or []
        self.verify_ssl = verify_ssl
        
    def post(
        self, 
        path: str, 
        headers: Dict[str, str], 
        payload: Dict[str, Any],
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if all retries failed
        """
        return self._post_with_retry(
            path=path,
            headers=headers,
            payload=payload,
            retries=self.max_retries,
            timeout=timeout or self.long_timeout
        )
        
    async def async_post(
        self, 
        path: str, 
        headers: Dict[str, str], 
        payload: Dict[str, Any],
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if all retries failed
        """
        return await self._async_post_with_retry(
            path=path,
            headers=headers,
            payload=payload,
            retries=self.max_retries,
            timeout=timeout or self.long_timeout
        )
        
    def get(
        self, 
        path: str, 
        headers: Dict[str, str],
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a GET request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.get(
                url,
                headers=headers,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout
            )
        except Exception as e:
            logger.error(f"GET request to {url} failed: {str(e)}")
            return None
            
    async def async_get(
        self, 
        path: str, 
        headers: Dict[str, str],
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous GET request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with aiohttp.ClientSession() as session:
                return await session.get(
                    url,
                    headers=headers,
                    timeout=timeout or self.default_timeout
                )
        except Exception as e:
            logger.error(f"Async GET request to {url} failed: {str(e)}")
            return None
            
    def delete(
        self, 
        path: str, 
        headers: Dict[str, str],
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a DELETE request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.delete(
                url,
                headers=headers,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout
            )
        except Exception as e:
            logger.error(f"DELETE request to {url} failed: {str(e)}")
            return None
            
    async def async_delete(
        self, 
        path: str, 
        headers: Dict[str, str],
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous DELETE request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with aiohttp.ClientSession() as session:
                return await session.delete(
                    url,
                    headers=headers,
                    timeout=timeout or self.default_timeout
                )
        except Exception as e:
            logger.error(f"Async DELETE request to {url} failed: {str(e)}")
            return None
        
    def _post_with_retry(
        self, 
        path: str, 
        headers: Dict[str, str], 
        payload: Dict[str, Any], 
        retries: int = 5, 
        timeout: int = 300
    ) -> Optional[requests.Response]:
        """
        Perform an HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        
        while remaining_retries > 0:
            try:
                response = requests.post(
                    url,
                    headers=headers,
                    json=payload,
                    verify=self.verify_ssl,
                    timeout=timeout,
                )
                
                # Check for HTTP errors
                if response.status_code >= 300:
                    detail = f"Error code from API: {response.status_code} {response.text}"
                    logger.error(f"Request failed: {detail}")
                else:
                    # Check for known error messages that might be in a 200 OK response
                    try:
                        response_json = response.json()
                        if (
                            "answer" in response_json 
                            and any(msg in response_json["answer"] for msg in self.known_error_messages)
                        ):
                            detail = f"Found known error message in response: {response_json['answer']}"
                            logger.debug(f"Backing off calling URL {url}: {detail}")
                        else:
                            return response
                    except ValueError:
                        # Not JSON or invalid JSON, but still a success status code
                        return response
            except Exception as e:
                logger.error(f"Exception during request to {url}: {str(e)}")
            
            # If we get here, we need to retry
            logger.info(f"Retrying {url} in {self.retry_delay} seconds. Attempts remaining: {remaining_retries-1}")
            time.sleep(self.retry_delay)
            remaining_retries -= 1
            
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        return None

    async def _async_post_with_retry(
        self, 
        path: str, 
        headers: Dict[str, str], 
        payload: Dict[str, Any], 
        retries: int = 5, 
        timeout: int = 300
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        error = None
        
        while remaining_retries > 0:
            try:
                async with aiohttp.ClientSession() as session:
                    response = await session.post(
                        url, 
                        headers=headers, 
                        json=payload,
                        timeout=timeout
                    )
                    
                    # Check for HTTP errors
                    if response.status >= 300:
                        detail = f"Error code from API: {response.status} {await response.text()}"
                        logger.error(f"Request failed: {detail}")
                    else:
                        # Check for known error messages in response
                        try:
                            result = await response.json()
                            if (
                                "answer" in result 
                                and any(msg in result["answer"] for msg in self.known_error_messages)
                            ):
                                detail = f"Found known error message in response: {result['answer']}"
                                logger.debug(f"Backing off calling URL {url}: {detail}")
                            else:
                                return response
                        except ValueError:
                            # Not JSON or invalid JSON, but still a success status code
                            return response
            except Exception as e:
                error = e
                logger.error(
                    f"Error occurred calling API {url}. Details: {str(e)}. "
                    f"Current retry attempt# {retries - remaining_retries + 1} of {retries}."
                )
                
            # If we get here, we need to retry
            logger.info(f"Sleeping for {self.retry_delay} seconds before next retry")
            await asyncio.sleep(self.retry_delay)
            remaining_retries -= 1
            
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        if error:
            logger.error(f"Last error: {str(error)}")
        return None